In [3]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())
!pip install tqdm
!pip install -U -q PyDrive
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

0.4.0
True


In [100]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

('Gen RAM Free: 8.9 GB', ' | Proc size: 3.5 GB')
GPU RAM Free: 3305MB | Used: 8134MB | Util  71% | Total 11439MB


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [87]:
def download_from_drive(drive, drive_folder_id = '1WyahQyFk_k8B94o092OwWl7ZDsaMTtek', dstn_path = '~/DreamPhant/dream/input'):
  # choose a local (colab) directory to store the data.
  local_download_path = os.path.expanduser(dstn_path)
  try:
    os.makedirs(local_download_path)
  except: pass

  file_list = drive.ListFile(
      {'q': "'{}' in parents".format(drive_folder_id)}).GetList()

  for f in file_list:
    # 3. Create & download by id.
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = os.path.join(local_download_path, f['title'])
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)

download_from_drive(drive = drive, drive_folder_id = '1WyahQyFk_k8B94o092OwWl7ZDsaMTtek', dstn_path = '~/DreamPhant/dream/input')
download_from_drive(drive = drive, drive_folder_id = '1JsoMCfY5WrwPPkLxR0pmG2OIVs7JTB4m', dstn_path = '~/DreamPhant/dream/guides')


title: 20180419_160143.jpg, id: 1_WtXaoJ9EzGz80_JDSpwOv_KjhjRUWPX
downloading to /content/DreamPhant/dream/input/20180419_160143.jpg
title: abstract-house.jpg, id: 1mW5xQnN-eKoOc4EsrmsL0BF-RGPZBIiX
downloading to /content/DreamPhant/dream/guides/abstract-house.jpg
title: abstract-horse.jpg, id: 1n7xy__Z2cM6_HDV-jeLzd8ZGmqP0LfIj
downloading to /content/DreamPhant/dream/guides/abstract-horse.jpg
title: abstract_owl.jpg, id: 1yVl7NtK4Xzav-ajoGrqX0zyfrW6mEs1F
downloading to /content/DreamPhant/dream/guides/abstract_owl.jpg
title: flower_bird.jpg, id: 1FuV4vItJnKpGEzHLrsCKkVExnflXoCmW
downloading to /content/DreamPhant/dream/guides/flower_bird.jpg
title: roses.jpg, id: 1gZ2WTOR3qtqxzC_uZ2uerSU28_t0LiUf
downloading to /content/DreamPhant/dream/guides/roses.jpg


In [0]:
_author__ = 'MSteger'

import numpy as np
import torch
import PIL
import os, gc
import scipy.ndimage as nd
import PIL.Image
from torch.autograd import Variable
from torchvision import transforms, models
from tqdm import tqdm

class DreamPhant(object):

    def __init__(self, model, input_dir, device, step_fn=None, verbose = True):
        self.model = model.to(device)
        self.input_dir = input_dir
        self.device = device
        self.step_fn = self.make_step if step_fn is None else step_fn
        self.verbose = verbose

    def _load_image(self, path, preprocess, resize = None):
        img = PIL.Image.open(path)
        if resize is not None: img.thumbnail(resize, PIL.Image.ANTIALIAS)
        img_tensor = preprocess(img).unsqueeze(0) if preprocess is not None else transforms.ToTensor(img)
        return img, img_tensor, img_tensor.numpy()

    def _data_to_img(self, t, tensor = True):
        if tensor: t = t.numpy()
        mean = np.array([0.485, 0.456, 0.406]).reshape([1, 1, 3])
        std = np.array([0.229, 0.224, 0.225]).reshape([1, 1, 3])
        inp = t[0, :, :, :]
        inp = inp.transpose(1, 2, 0)
        inp = std * inp + mean
        inp *= 255
        inp = np.uint8(np.clip(inp, 0, 255))
        return PIL.Image.fromarray(inp)

    def _image_to_variable(self, image, requires_grad=False):
        return Variable(image.cuda() if self.device == torch.device('cuda') else image, requires_grad=requires_grad)

    def _extract_features(self, img_tensor, layer, model = None):
        if model is None: model = self.model
        features = self._image_to_variable(img_tensor, requires_grad=True) if not isinstance(img_tensor, (torch.cuda.FloatTensor if self.device == torch.device('cuda')  else torch.Tensor)) else img_tensor
        for index, current_layer in enumerate(model.features.children()):
            features = current_layer(features)
            if index == layer: break
        return features

    def objective(self, dst, guide_features=None):
        if guide_features is None:
            return dst.data
        else:
            x = dst.data[0].cpu().numpy()
            y = guide_features.data[0].cpu().numpy()
            ch, w, h = x.shape
            x = x.reshape(ch, -1)
            y = y.reshape(ch, -1)
            A = x.T.dot(y)
            diff = y[:, A.argmax(1)]
            return torch.Tensor(np.array([diff.reshape(ch, w, h)])).to(self.device)

    def make_step(self, img, control=None, step_size=1.5, layer=28, jitter=32):

        mean = np.array([0.485, 0.456, 0.406]).reshape([3, 1, 1])
        std = np.array([0.229, 0.224, 0.225]).reshape([3, 1, 1])

        ox, oy = np.random.randint(-jitter, jitter + 1, 2) # offset by random jitter

        img = np.roll(np.roll(img, ox, -1), oy, -2) # apply jitter shift
        tensor = torch.Tensor(img)

        img_var = self._image_to_variable(tensor, requires_grad=True)
        self.model.zero_grad()

        x = self._extract_features(img_tensor=img_var, layer=layer)
        delta = self.objective(x, control)
        x.backward(delta)

        # L2 Regularization on gradients
        mean_square = torch.Tensor([torch.mean(img_var.grad.data ** 2)]).to(self.device)
        img_var.grad.data /= torch.sqrt(mean_square)
        img_var.data.add_(img_var.grad.data * step_size)

        result = img_var.data.cpu().numpy()
        result = np.roll(np.roll(result, -ox, -1), -oy, -2)
        result[0, :, :, :] = np.clip(result[0, :, :, :], -mean / std, (1 - mean) / std) # normalize img

        return torch.Tensor(result)

    def DeepDream(self, base_img, octave_n=6, octave_scale=1.4, iter_n=10, **step_args):
        octaves = [base_img]
        for i in range(octave_n - 1): octaves.append(nd.zoom(octaves[-1], (1, 1, 1.0 / octave_scale, 1.0 / octave_scale), order=1))

        detail = np.zeros_like(octaves[-1])

        for octave, octave_base in enumerate(octaves[::-1]):
            h, w = octave_base.shape[-2:]
            if octave > 0:
                h1, w1 = detail.shape[-2:]
                detail = nd.zoom(detail, (1, 1, 1.0 * h / h1, 1.0 * w / w1), order=1)
            src = octave_base + detail
            for i in range(iter_n):
                src = self.step_fn(src, **step_args)

            detail = src.numpy() - octave_base

        return src

    def transform(self, preprocess, layer, control=None, resize = [1024, 1024], repeated = 10, file_prefix = None,**dream_args):
        if (repeated is None) | (repeated is False): repeated = 1
        if control is not None:
            _, guideImage_tensor, _ = self._load_image(path=control[2], preprocess=control[3], resize=control[4])
            control = self._extract_features(img_tensor=guideImage_tensor, layer = control[0])
        for img_name in os.listdir(self.input_dir):
            img_path = os.path.join(self.input_dir, img_name)
            _, _, frame = self._load_image(path=img_path, preprocess=preprocess, resize=resize)
            bar = tqdm(total=repeated, unit='iteration')
            for i in range(repeated):
                frame = self.DeepDream(base_img=frame, layer = layer, control = control, **dream_args).numpy()
                bar.update(1)
                bar.set_description('Processing Image: {}'.format(img_name))
            bar.close()
            DeepDream = self._data_to_img(frame, tensor=False)
            output_dir = os.path.join(self.input_dir.replace('/input', '/output'), 'layer{}'.format(layer))
            if not os.path.exists(output_dir): os.makedirs(output_dir)
            if file_prefix is not None: img_name = '{}_{}'.format(file_prefix, img_name)
            output_path = os.path.join(output_dir, img_name)
            DeepDream.save(output_path)
            if self.verbose: print 'saved img {} to {}'.format(os.path.split(img_name)[-1], output_path)
        return self

In [0]:
class summary(object):

    def __init__(self, model, device, input_size=(1,1,256,256), verbose = True):
        self.model = model.to(device)
        self.input_size = input_size
        self.device = device
        self.iterate()
        if verbose: self.printer()

    def compute_output(self, input, layer):
        if isinstance(layer, torch.nn.Linear):
            try:
                input = input.resize(1, layer.in_features)
            except Exception as e:
                print 'Failure! {} >> using hack'.format(e)
                input = input.mean(-1).mean(-1) # ...
        return layer(input)

    def compute_no_params(self, layer):
        params_to_optim, params_frozen = [], []
        for p in layer.parameters():
            if p.requires_grad:
                params_to_optim.append(tuple(p.size()))
            else:
                params_frozen.append(tuple(p.size()))
        return [np.sum([np.prod(p) for p in params_to_optim]).astype(int), np.sum([np.prod(p) for p in params_frozen]).astype(int)]

    def iterate(self):
        summary = []
        with torch.no_grad():
            input = torch.autograd.Variable(torch.FloatTensor(*self.input_size)).to(self.device)
            for k, v in self.model._modules.iteritems():
                if isinstance(v, torch.nn.Sequential):
                    for layer in v:
                        output = self.compute_output(input, layer)
                        summary.append([k, type(layer).__name__, tuple(input.shape)[1:], tuple(output.shape)[1:]] + self.compute_no_params(layer))
                        input = output
                else:
                    layer = v
                    output = self.compute_output(input, layer)
                    summary.append([k, type(layer).__name__, tuple(input.shape)[1:], tuple(output.shape)[1:]] + self.compute_no_params(layer))
                    input = output
        self.summary = summary
        return summary

    def printer(self, summary = None):
        if summary is None: summary = self.summary
        total_params, trainable_params = 0, 0
        print 'Model Summary'
        print '---------------------------------------------------------------------------------------------------------------------------------'
        print '{:>2} {:>20} {:>20} {:>20} {:>20} {:>20} {:>20}'.format('Id', 'Name', 'Type', 'Input', 'Output', 'Params', 'Params(Frozen)')
        print '---------------------------------------------------------------------------------------------------------------------------------'
        for idx, layer in enumerate(summary):
            print '{:>2} {:>20}  {:>20} {:>20} {:>20} {:>20} {:>20}'.format(*[idx]+layer)
            total_params += layer[-2] + layer[-1]
            trainable_params += layer[-2]
        print '================================================================================================================================='
        print 'Total params: {0:,}'.format(total_params)
        print 'Trainable params: {0:,}'.format(trainable_params)
        print 'Non-trainable params: {0:,}'.format(total_params - trainable_params)
        print '---------------------------------------------------------------------------------------------------------------------------------'

        return self

In [97]:
# setup
input_dir = r'/content/DreamPhant/dream/input/'
guideImage_dir = r'/content/DreamPhant/dream/guides/'
preprocess = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
preprocess_resize = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
device = torch.device('cuda')
layer, layer_guide = 34, 34
iter_n = 5
step_size = 0.01
jitter = 32
guided = True

# model
model = models.vgg19(pretrained=True)
for weights in model.parameters(): weights.requires_grad = False
summary(model=model, device=device, input_size=(1,3,224,224))

Model Summary
---------------------------------------------------------------------------------------------------------------------------------
Id                 Name                 Type                Input               Output               Params       Params(Frozen)
---------------------------------------------------------------------------------------------------------------------------------
 0             features                Conv2d        (3, 224, 224)       (64, 224, 224)                    0                 1792
 1             features                  ReLU       (64, 224, 224)       (64, 224, 224)                    0                    0
 2             features                Conv2d       (64, 224, 224)       (64, 224, 224)                    0                36928
 3             features                  ReLU       (64, 224, 224)       (64, 224, 224)                    0                    0
 4             features             MaxPool2d       (64, 224, 224)       (64,

In [98]:
# dreaming
for guideImage_name in os.listdir(guideImage_dir):
    print guideImage_name
    guideImage_path = os.path.join(guideImage_dir, guideImage_name)
    guideImage_name = guideImage_name.split('.jpg')[0]
    for rep in [30,60,90]:#range(1, 2, 1):
        Dream = DreamPhant(model=model, input_dir=input_dir, device=device) #
        Dream.transform(preprocess = preprocess, resize = [768, 1024], layer = layer, octave_n=6, octave_scale=1.4,iter_n=iter_n, control=(model, layer_guide, guideImage_path, preprocess_resize, None) if guided else None,\
                        step_size=step_size, jitter=jitter, repeated = rep, file_prefix='{}_{}_{}_{}_{}_{}'.format(guideImage_name, rep, iter_n, step_size, jitter, guided))
        Dream = None
        gc.collect()


  0%|          | 0/30 [00:00<?, ?iteration/s]

roses.jpg



Processing Image: 20180419_160143.jpg:  43%|████▎     | 13/30 [01:24<01:50,  6.50s/iteration]


Processing Image: 20180419_160143.jpg:  87%|████████▋ | 26/30 [02:49<00:25,  6.49s/iteration]


Processing Image: 20180419_160143.jpg: 100%|██████████| 30/30 [03:15<00:00,  6.50s/iteration]

  0%|          | 0/60 [00:00<?, ?iteration/s]

...


Processing Image: 20180419_160143.jpg:  58%|█████▊    | 52/90 [05:39<04:07,  6.52s/iteration]


Processing Image: 20180419_160143.jpg:  72%|███████▏  | 65/90 [07:04<02:43,  6.53s/iteration]


Processing Image: 20180419_160143.jpg:  87%|████████▋ | 78/90 [08:28<01:18,  6.53s/iteration]


Processing Image: 20180419_160143.jpg: 100%|██████████| 90/90 [09:47<00:00,  6.53s/iteration]


saved img abstract_owl_90_5_0.01_32_True_20180419_160143.jpg to /content/DreamPhant/dream/output/layer34/abstract_owl_90_5_0.01_32_True_20180419_160143.jpg


In [103]:
# MAYBE you have to rerun the authentification process (3rd cell)
# from functools import partial
from google.colab import files

def upload_to_drive(path, drive):
  pathBody, filename = os.path.split(path)
  parentDir = pathBody.split('/')[-1]
  uploaded = drive.CreateFile({'title': '{}_{}'.format(parentDir, filename)})
  uploaded.SetContentFile(path)
  uploaded.Upload()
  print 'file >> {} to drive >> with ID: {}'.format(path,uploaded.get('id'))

def test(path):
  print path

def walkDir(path, upload=False):   
  # f**k jupyter notebooks...
  for f in os.listdir(path):
    pathname = os.path.join(path, f)
    if os.path.isdir(pathname):
      walkDir(pathname)
    elif os.path.isfile(pathname):
      print 'file >> {}'.format(pathname)
      if not upload: continue
      upload_to_drive(pathname, drive)
#       download_file(pathname)
    else:
      print 'Skipping {}'.format(pathname)

def download_file(path):
  return files.download(path)

folder = r'/content/DreamPhant/dream/output/layer34'
walkDir(path=folder,upload=True)

file >> /content/DreamPhant/dream/output/layer34/roses_30_5_0.01_32_True_20180419_160143.jpg
file >> /content/DreamPhant/dream/output/layer34/roses_30_5_0.01_32_True_20180419_160143.jpg to drive >> with ID: 19Nx34KfvpNg2KxCFIWB7PZY3OYkgRons
file >> /content/DreamPhant/dream/output/layer34/roses_1_50_0.01_32_True_20180419_160143.jpg
file >> /content/DreamPhant/dream/output/layer34/roses_1_50_0.01_32_True_20180419_160143.jpg to drive >> with ID: 1JfxVBLWkqeAD6GgjvBoNAvabPRYDh0n_
file >> /content/DreamPhant/dream/output/layer34/abstract-horse_30_5_0.01_32_True_20180419_160143.jpg
file >> /content/DreamPhant/dream/output/layer34/abstract-horse_30_5_0.01_32_True_20180419_160143.jpg to drive >> with ID: 1sOVkhXuXvosObSUHZpj4H5qsMYv7bcr3
file >> /content/DreamPhant/dream/output/layer34/flower_bird_90_5_0.01_32_True_20180419_160143.jpg
file >> /content/DreamPhant/dream/output/layer34/flower_bird_90_5_0.01_32_True_20180419_160143.jpg to drive >> with ID: 1ibv4edFLxLwEBeM5eIUbtWMP64A5qVnh
file >

In [90]:
!ls ~/DreamPhant/dream/output/layer30/
# !rm -rf ~/DreamPhant/dream/output/layer30/
# !ls ~/DreamPhant/dream/output/layer30/


ls: cannot access '/content/DreamPhant/dream/output/layer30/': No such file or directory
